In [4]:
import numpy as np
import pandas as pd
from datetime import timedelta
import seaborn as sns
import matplotlib.pylab as plt

plt.rcParams['font.sans-serif'] = ['SimHei'] #设置画图的中文字体显示
plt.rcParams['axes.unicode_minus'] = False   #显示负号
plt.rcParams['font.size'] = 12

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

#隐藏红色警示框
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

seed = 123

In [2]:
original_df = pd.read_csv('final_data.csv')
original_huayan_df = pd.read_excel('huayan_data.xlsx')

In [5]:
control = ['给料皮带G1_矿量反馈', '一段泵池_补加水流量反馈', '一段渣浆泵1_矿浆流量反馈', '球磨机_排矿水阀位']
environment = ['半自磨_功率2', '半自磨_出料端总高压', '一段旋流器1_压力', '球磨机_出料端总高压', '1系列1段旋流器A组溢流浓度/%']
features = original_df[control + environment]
target = original_df['1系列1段旋流器A组溢流粒度/-200目']
Xtrain,Xtest,Ytrain,Ytest = train_test_split(features,target,test_size = 0.3,random_state=seed)

In [7]:
forest = RandomForestRegressor(random_state=seed)
forest.fit(Xtrain,Ytrain)

rfr_score = forest.score(Xtest, Ytest)
print('rfr_score:',rfr_score.round(3))
rfr_rmse = metrics.mean_squared_error(Ytest,forest.predict(Xtest)) ** 0.5 # ,squared=False
print('rfr_rmse:',rfr_rmse.round(3))

rfr_score: 0.84
rfr_rmse: 0.232


### L-BFGS-B

In [8]:
def func(x):
    inputs = [x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]]
    inputs = np.array(inputs).reshape(1, -1)
    outputs = forest.predict(inputs)
    return outputs

In [9]:
bounds=[[900,1100],[2900,4000],[2900,3500],[99,100],[3000,5000],[800,950],[0.05,1],[55,60],[17,25]]
x0 = [950,3000,3200,99.5,4000,900,0.07,57,20]
res = minimize(fun=func, x0=x0, bounds=bounds, method='L-BFGS-B')
print(res.x)
print(res.fun)

[9.50e+02 3.00e+03 3.20e+03 9.95e+01 4.00e+03 9.00e+02 7.00e-02 5.70e+01
 2.00e+01]
[88.7202]


### L-BFGS-B.Example

In [10]:
def func(x, sign, c):    
    return sign * (x[0] ** 2 + x[1] + x[2] ** 2) + c

bounds=[[1,2],[1,3],[1,2]]
mini = minimize(fun=func, x0=[2,3,1], args=(2, 1), bounds=bounds, method='SLSQP')
print(mini.x)
print(mini.fun)

[1. 1. 1.]
7.000000000000046
